In [129]:
# !pip install scikit-learn==1.5.2
# !pip install lets-plot

In [130]:
import pandas as pd
from lets_plot import *
from sklearn.preprocessing import MinMaxScaler

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

In [131]:
# Step 1: Remove the time portion chat gpt code. I only turned many new column fucntions into a single assign funciton
housing['datetime'] = housing['date'].str.split('T').str[0]

# Step 2: Extract year, month, day from compact YYYYMMDD format
housing = housing.assign(
    year = housing['datetime'].str[0:4].astype(int),
    month = housing['datetime'].str[4:6].astype(int),
    day = housing['datetime'].str[6:8].astype(int)
)
# Drop the intermediate 'date' column if desired
housing = housing.drop(columns=['date'])
housing = housing.drop(columns=['datetime'])

In [132]:
clean_house = housing
clean_house = clean_house[clean_house['bedrooms'] <= 15]
clean_house = clean_house[(clean_house['bathrooms'] <= 5.5) & (clean_house['bathrooms'] >= .5)]
clean_house = clean_house[clean_house['sqft_living'] <= 8670]
clean_house = clean_house[clean_house['sqft_living'] <= 450000]
clean_house = clean_house[clean_house['floors'] <= 3]
clean_house = clean_house[clean_house['sqft_above'] <= 6500]
clean_house = clean_house[clean_house['sqft_basement'] <= 2750]

scaler = MinMaxScaler()

# numeric_cols = clean_house.select_dtypes(include='number').columns
# numeric_cols
cols_to_convert = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated',
       'sqft_living15', 'sqft_lot15']
clean_house[cols_to_convert] = scaler.fit_transform(clean_house[cols_to_convert])


In [133]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [134]:
['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'price', 'year', 'month', 'day']

X = clean_house[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month', 'day']]

y = clean_house['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
model = XGBRegressor()
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [136]:
predictions = model.predict(X_test)
predictions

array([779944.5 , 408676.75, 189618.58, ..., 445424.2 , 483735.47,
       745531.1 ], dtype=float32)

In [137]:
from sklearn.metrics import root_mean_squared_error

result = root_mean_squared_error(y_test, predictions)
result

np.float64(115061.19682549583)

In [138]:
# (
#     ggplot()
# )115061.19682549583